In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp
import scipy.stats as stats
import scipy.optimize as opt
import imageio
import utils

def approx_mean(xs, ys):
    return xs*0 + ys * 0

def kernel_fit(l, sigma, ls):
    return sigma**2 * np.exp(-(l**2)/(2*ls**2))


disc_res = 2

xrange = (-1,1)
yrange = (-1,1)

disc_x = np.linspace(*xrange, disc_res)
disc_y = np.linspace(*yrange, disc_res)

kernel_params_x = np.array([
    [1,1],
    [1,1],
    [1,1],
    [1,1],
])

kernel_params_x_interp = interp.RegularGridInterpolator((disc_x, disc_y), kernel_params_x, "linear", False, None)

def k(x0,y0,x1,y1):
    p0x = np.moveaxis(kernel_params_x_interp((x0,y0)), -1, 0)
    p1x = np.moveaxis(kernel_params_x_interp((x1,y1)), -1, 0)

    p0x = np.maximum(p0x, 0.00001)
    p1x = np.maximum(p1x, 0.00001)

    dx = x0-x1
    dy = y0-y1

    detAnisoA = p0x[1]**2
    detAnisoB = p1x[1]**2

    anisoABavg = 0.5 * (p0x[1] + p1x[1])
    detAnisoABavg = anisoABavg**2
    ansioFac = (detAnisoA**0.25 * detAnisoB**0.25) / np.sqrt(detAnisoABavg)

    anisoABavgInv = (1/anisoABavg)
    ln = np.sqrt(dx*anisoABavgInv*dx + dy*anisoABavgInv*dy)

    return np.sqrt(p0x[0]) * np.sqrt(p1x[0]) * ansioFac * kernel_fit(ln, 1, 1)
                
    #return 0.5*(kernel_fit(dx, *p0x) * kernel_fit(dy, *p0y) + 
    #            kernel_fit(dx, *p1x) * kernel_fit(dy, *p1y))


def mean_and_cov(xs,ys):
    M = approx_mean(xs, ys)
    x0s, x1s = np.meshgrid(xs, xs)
    y0s, y1s = np.meshgrid(ys, ys)
    K = k(x0s, y0s, x1s, y1s)
    return M, K


real_res = 50
real_x = np.linspace(*xrange, real_res)
real_y = np.linspace(*yrange, real_res)
real_xx, real_yy = np.meshgrid(real_x, real_y)

M, K = mean_and_cov(real_xx.ravel(), real_yy.ravel())

Kimg = K.reshape(real_res*real_res, real_res*real_res)
plt.show()
plt.imshow(Kimg)
plt.show()

dist = stats.multivariate_normal(M, K, allow_singular=True)
samples = dist.rvs(4)

fig, axs = plt.subplots(1, samples.shape[0], figsize=(samples.shape[0]*4, 4))
for i in range(samples.shape[0]):
    utils.plot_realization(samples[i].reshape(real_res, real_res), axs[i], real_xx, real_yy)


ValueError: There are 2 points and 4 values in dimension 0

In [ ]:

fig_occ, axs_occ = plt.subplots(1,4, figsize=(16,4))
axs_occ[0].set_title("downsampled mean (linear interp)")
utils.plot_realization(approx_mean(xx,yy), axs_occ[0], xx, yy)
axs_occ[0].grid(True, "major", "both")
axs_occ[0].set_xticks(disc_x)
axs_occ[0].set_yticks(disc_y)

axs_occ[1].set_title("occupancy based on downsampled mean")
utils.plot_realization( -((interp_mean < 0) - 0.5), axs_occ[1], xx, yy)
axs_occ[1].grid(True, "major", "both")
axs_occ[1].set_xticks(disc_x)
axs_occ[1].set_yticks(disc_y)

axs_occ[2].set_title("downsampled occupancy (linear interp)")
utils.plot_realization(grid_occ_interp((xx,yy)), axs_occ[2], xx, yy)
axs_occ[2].grid(True, "major", "both")
axs_occ[2].set_xticks(disc_x)
axs_occ[2].set_yticks(disc_y)
var = k(real_xx.ravel(), real_yy.ravel(), real_xx.ravel(), real_yy.ravel())
ptwise_occupancy = stats.norm.cdf(np.zeros_like(M), loc=M, scale=var*100+0.0001) - 0.5
axs_occ[3].set_title("Pointwise GP Occupancy")
utils.plot_realization(-ptwise_occupancy.reshape(real_res, real_res), axs_occ[3], real_xx, real_yy)
fig_occ.tight_layout()